In [2]:
import pandas as pd
import torch

In [3]:
import numpy as np

In [4]:
ds = pd.read_excel('./data/lequipe2017_18.xlsx')

In [5]:
ds.head()

,id,url,source,title,h1,chapeau,text,detectiondate,crawlurldate,crawldate,publicationdate
0,104,https://www.lequipe.fr/Football/Actualites/Wen...,https://www.lequipe.fr,Wendie Renard (Bleues) : « On a encore beaucou...,Wendie Renard (Bleues) : « On a encore beaucou...,"Wendie Renard, la défenseure de l'équipe de Fr...",Wendie Renard (défenseure de l'équipe de Franc...,2018-10-10T10:56:33+0200,2018-10-10T11:21:19+0200,2018-10-10T15:58:36+0200,2018-10-09T23:49:00+0200
1,106,https://www.lequipe.fr/Football/Actualites/Ami...,https://www.lequipe.fr,Amical : Les Bleues réorganisées mais bien sup...,Amical : Les Bleues réorganisées mais bien sup...,Face à une équipe camerounaise vite essoufflée...,Il n'y avait pas vraiment de suspense avant la...,2018-10-10T10:56:33+0200,2018-10-10T11:22:17+0200,2018-10-10T15:59:06+0200,2018-10-09T22:55:00+0200
2,107,https://www.lequipe.fr/Football/Actualites/Eug...,https://www.lequipe.fr,Eugénie Le Sommer (Bleues) : «J'ai essayé de f...,Eugénie Le Sommer (Bleues) : «J'ai essayé de f...,"Eugénie Le Sommer, habituelle attaquante de l'...",Eugénie Le Sommer (attaquante de l'équipe de F...,2018-10-10T10:56:33+0200,2018-10-10T11:22:41+0200,2018-10-10T15:59:45+0200,2018-10-10T00:11:00+0200
3,109,https://www.lequipe.fr/Football/Actualites/Dim...,https://www.lequipe.fr,Dimitri Payet remplace Nabil Fekir en équipe d...,Dimitri Payet remplace Nabil Fekir en équipe d...,"Blessé à la cheville gauche, Nabil Fekir (OL) ...",NaN,2018-10-10T10:56:33+0200,2018-10-10T11:23:47+0200,2018-10-10T16:00:15+0200,2018-10-08T12:01:00+0200
4,110,https://www.lequipe.fr/Football/Actualites/Tou...,https://www.lequipe.fr,"Touché à une cheville, Fekir sort contre le PS...","Touché à une cheville, Fekir sort contre le PSG",Touché à une cheville sur un tacle de Thiago S...,Le début de match de Lyon à Paris est cauchema...,2018-10-10T10:56:33+0200,2018-10-10T11:24:12+0200,2018-10-10T16:00:43+0200,2018-10-07T21:14:00+0200


In [4]:
ds.loc[1, 'chapeau']

"Face à une équipe camerounaise vite essoufflée, l'équipe de France féminine, dans un nouveau schéma tactique, a fait le show ce mardi en amical (6-0), au stade des Alpes de Grenoble. Sixième succès consécutif pour Corinne Diacre et ses joueuses."

## Camembert

In [19]:
from transformers import CamembertModel, CamembertTokenizer

# You can replace "camembert-base" with any other model from the table, e.g. "camembert/camembert-large".
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
camembert = CamembertModel.from_pretrained("camembert-base")

In [2]:
camembert.eval()  # disable dropout (or leave in train mode to finetune)

CamembertModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.1, in

In [28]:
encoded_sentence = tokenizer.encode(ds.loc[1, 'chapeau'])
encoded_sentence
encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)
embeddings, _ = camembert(encoded_sentence)

In [29]:
embeddings

tensor([[[-0.0023,  0.1262,  0.0032,  ..., -0.0521,  0.1549,  0.0195],
         [-0.0612,  0.0019,  0.0662,  ...,  0.1292,  0.0873, -0.0180],
         [ 0.0417,  0.0743,  0.0451,  ..., -0.0206,  0.1984, -0.0205],
         ...,
         [ 0.0375,  0.1274, -0.1516,  ..., -0.0278,  0.0876,  0.0209],
         [-0.0559,  0.1097,  0.0200,  ..., -0.0408,  0.1030,  0.0897],
         [-0.0679,  0.1712,  0.0011,  ..., -0.0095,  0.1073, -0.0083]]],
       grad_fn=<NativeLayerNormBackward>)

## BARThez

In [1]:
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)

barthez_tokenizer = AutoTokenizer.from_pretrained('moussaKam/barthez')
barthez_model = AutoModelForSeq2SeqLM.from_pretrained('moussaKam/barthez-orangesum-abstract')

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/557M [00:00<?, ?B/s]

In [26]:
text_sentence  = ds.loc[80, 'text']
text_sentence 

"«\xa0Ce qui nous empêche d'attaquer\xa0? Je ne peux pas vous dire. Je ne suis pas l'entraîneur.\xa0»La sortie de Paul Pogba, fin septembre, après le match nul (1-1) à domicile face à Wolverhampton avait fait beaucoup de bruit, d'autant que les rapports entre Pogba et son entraîneur, José Mourinho, ne sont pas au beau fixe.\n                    \n                                                    \n                                                      \n                                              \n                                                                                                                                                                                                        \n                        Louis Saha, l'ancien attaquant de Manchester United entre\xa02004 et\xa02008, n'a pas apprécié les propos de son compatriote. «\xa0Je crois qu'il l'a fait exprès et je ne pense pas que ce soit correct, a confié Saha à Reuters.\xa0C'est une mauvaise communication, cel

In [47]:
def sum_sequence(text_sentence):
    if pd.isna(text_sentence):
        return np.nan
    elif len(text_sentence)>1000:
        return np.nan
    else:
        input_ids = torch.tensor([barthez_tokenizer.encode(text_sentence, add_special_tokens=True)])

        barthez_model.eval()
        predict = barthez_model.generate(input_ids, max_length=500)[0]

        return barthez_tokenizer.decode(predict, skip_special_tokens=True)

In [48]:
ds['text_sum'] = ds['text'].apply(sum_sequence)

In [49]:
ds.to_csv('./data/lequipe_scored.csv')

In [51]:
ds.head(50)

,id,url,source,title,h1,chapeau,text,detectiondate,crawlurldate,crawldate,publicationdate,text_sum
0,104,https://www.lequipe.fr/Football/Actualites/Wen...,https://www.lequipe.fr,Wendie Renard (Bleues) : « On a encore beaucou...,Wendie Renard (Bleues) : « On a encore beaucou...,"Wendie Renard, la défenseure de l'équipe de Fr...",Wendie Renard (défenseure de l'équipe de Franc...,2018-10-10T10:56:33+0200,2018-10-10T11:21:19+0200,2018-10-10T15:58:36+0200,2018-10-09T23:49:00+0200,NaN
1,106,https://www.lequipe.fr/Football/Actualites/Ami...,https://www.lequipe.fr,Amical : Les Bleues réorganisées mais bien sup...,Amical : Les Bleues réorganisées mais bien sup...,Face à une équipe camerounaise vite essoufflée...,Il n'y avait pas vraiment de suspense avant la...,2018-10-10T10:56:33+0200,2018-10-10T11:22:17+0200,2018-10-10T15:59:06+0200,2018-10-09T22:55:00+0200,NaN
2,107,https://www.lequipe.fr/Football/Actualites/Eug...,https://www.lequipe.fr,Eugénie Le Sommer (Bleues) : «J'ai essayé de f...,Eugénie Le Sommer (Bleues) : «J'ai essayé de f...,"Eugénie Le Sommer, habituelle attaquante de l'...",Eugénie Le Sommer (attaquante de l'équipe de F...,2018-10-10T10:56:33+0200,2018-10-10T11:22:41+0200,2018-10-10T15:59:45+0200,2018-10-10T00:11:00+0200,NaN
3,109,https://www.lequipe.fr/Football/Actualites/Dim...,https://www.lequipe.fr,Dimitri Payet remplace Nabil Fekir en équipe d...,Dimitri Payet remplace Nabil Fekir en équipe d...,"Blessé à la cheville gauche, Nabil Fekir (OL) ...",NaN,2018-10-10T10:56:33+0200,2018-10-10T11:23:47+0200,2018-10-10T16:00:15+0200,2018-10-08T12:01:00+0200,NaN
4,110,https://www.lequipe.fr/Football/Actualites/Tou...,https://www.lequipe.fr,"Touché à une cheville, Fekir sort contre le PS...","Touché à une cheville, Fekir sort contre le PSG",Touché à une cheville sur un tacle de Thiago S...,Le début de match de Lyon à Paris est cauchema...,2018-10-10T10:56:33+0200,2018-10-10T11:24:12+0200,2018-10-10T16:00:43+0200,2018-10-07T21:14:00+0200,Le début de match de Lyon est très compliqué p...
5,112,https://www.lequipe.fr/Football/Actualites/Rob...,https://www.lequipe.fr,"Robert Pirès, VRP de Thierry Henry à Aston Vil...","Robert Pirès, VRP de Thierry Henry à Aston Villa",Thierry Henry a été érigé en priorité pour pre...,"Alors que, depuis ce mardi, Thierry Henry est ...",2018-10-10T10:56:33+0200,2018-10-10T11:25:16+0200,2018-10-10T16:01:15+0200,2018-10-10T00:57:00+0200,NaN
6,114,https://www.lequipe.fr/Football/Actualites/Mon...,https://www.lequipe.fr,Monaco : Leonardo Jardim vers la sortie - Foot...,Monaco : Leonardo Jardim vers la sortie,Fragilisé par les résultats catastrophiques de...,Au club depuis juin 2014 et sous contrat jusqu...,2018-10-10T10:56:33+0200,2018-10-10T11:28:57+0200,2018-10-10T16:01:41+0200,2018-10-09T17:26:00+0200,Le club de football de l'AS Monaco a écarté Le...
7,119,https://www.lequipe.fr/Football/Actualites/Did...,https://www.lequipe.fr,Équipe de France : Didier Deschamps a parlé av...,Équipe de France : Didier Deschamps a parlé av...,"Rappelé en équipe de France, Lucas Digne a eu ...",Ils n'ont pas attendu longtemps pour se dire l...,2018-10-10T10:56:33+0200,2018-10-10T11:31:26+0200,2018-10-10T16:02:15+0200,2018-10-09T18:41:00+0200,NaN
8,120,https://www.lequipe.fr/Football/Actualites/Pau...,https://www.lequipe.fr,Paul Pogba (Equipe de France) : «C'est dur de ...,Paul Pogba (Equipe de France) : «C'est dur de ...,Après avoir «touché les étoiles» cet été au Mo...,« Une année post-Coupe du monde est-elle plus ...,2018-10-10T10:56:33+0200,2018-10-10T11:31:53+0200,2018-10-10T16:02:16+0200,2018-10-09T19:03:00+0200,NaN
9,121,https://www.lequipe.fr/Football/Actualites/Pau...,https://www.lequipe.fr,"Paul Pogba : « Griezmann, Mbappé et Varane mér...","Paul Pogba : « Griezmann, Mbappé et Varane mér...","Paul Pogba, le milieu de terrain de Manchester...","Comme Antoine Griezmann, Paul Pogba aimerait q...",2018-10-10T10:56:33+0200,2018-10-10T11:32:26+0200,2018-10-10T16:02:22+0200,2018-10-09T18:25:00+0200,NaN


### Fine tune 

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

In [10]:
train_id, test_id = train_test_split(np.arange(len(ds)), test_size=0.3)

In [19]:
ds.loc[train_id,['text','chapeau']].dropna().to_csv('./data/train.csv', index=False)
ds.loc[test_id,['text','chapeau']].dropna().to_csv('./data/test.csv', index=False)